# Del 6: Optimizacija kode za velike datasete

Pripravimo datasete:

In [2]:
!tar -xJf data/data_del_06.tar.xz -C ./data/

In [3]:
import pandas as pd
import numpy as np

Viri:
- [A Beginner’s Guide to Optimizing Pandas Code for Speed](https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6)
- [How to Optimize your Pandas Code](https://kanoki.org/2019/01/09/how-to-optimize-your-pandas-code/)
- [Optimization tricks](http://ehneilsen.net/notebook/pandasExamples/pandas_examples.html#orgheadline36)
- [Enhancing performance](https://pandas.pydata.org/pandas-docs/stable/user_guide/enhancingperf.html)
- [Fast, Flexible, Easy and Intuitive: How to Speed Up Your Pandas Projects](https://realpython.com/fast-flexible-pandas/)
- [Optimizing Code Performance On Large Datasets](https://app.dataquest.io/course/improving-code-performance)
- [4 Unique Methods to Optimize your Python Code for Data Science](https://www.analyticsvidhya.com/blog/2019/09/4-methods-optimize-python-code-data-science/)
- [High-Performance Pandas: eval() and query()](https://jakevdp.github.io/PythonDataScienceHandbook/03.12-performance-eval-and-query.html#pandas.eval()-for-Efficient-Operations)

**Code optimization, in simple terms, means reducing the number of operations to execute any task while producing the correct results.**

## CPU Bound Programs

### Bounds vs Limitations

<img alt="I/O bounds" src="images/CPU+and+I_O+bounds.png">

In [ ]:
#poznamo dvojne omejitve:
#1. CPU bound: omejtive pri procesu, pomenijo, kako hitro se naša koda izvaja; potimizacija kode pomaga
#2. I/O bound: input/output bound: zunanje omejitve, recimo ko beremo  vsakič iz enega csvja (to je počasnejše kot če bi 
#    imeli v pomnilniku); branje  iz zunanjih virov; hitrost odvisna od zunanjih dejavnikov; poskusimo čimveć stvari 
#shraniti v pomnilnik in čimmanjkrat rat iz zunanjoh virov


### Primer optimizacije

In [4]:
import numpy as np

# Define a basic Haversine distance formula
def haversine(lat1, lon1, lat2, lon2):
    MILES = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    total_miles = MILES * c
    return total_miles

In [5]:
df = pd.read_csv('data/new_york_hotels.csv')

In [6]:
df.head()

,ean_hotel_id,name,address1,city,state_province,postal_code,latitude,longitude,star_rating,high_rate,low_rate
0,269955,Hilton Garden Inn Albany/SUNY Area,1389 Washington Ave,Albany,NY,12206,42.68751,-73.81643,3.0,154.0272,124.0216
1,113431,Courtyard by Marriott Albany Thruway,1455 Washington Avenue,Albany,NY,12206,42.68971,-73.82021,3.0,179.0100,134.0000
2,108151,Radisson Hotel Albany,205 Wolf Rd,Albany,NY,12205,42.72410,-73.79822,3.0,134.1700,84.1600
3,254756,Hilton Garden Inn Albany Medical Center,62 New Scotland Ave,Albany,NY,12208,42.65157,-73.77638,3.0,308.2807,228.4597
4,198232,CrestHill Suites SUNY University Albany,1415 Washington Avenue,Albany,NY,12206,42.68873,-73.81854,3.0,169.3900,89.3900


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1631 entries, 0 to 1630
Data columns (total 11 columns):
ean_hotel_id      1631 non-null int64
name              1631 non-null object
address1          1631 non-null object
city              1631 non-null object
state_province    1631 non-null object
postal_code       1631 non-null object
latitude          1631 non-null float64
longitude         1631 non-null float64
star_rating       1630 non-null float64
high_rate         1631 non-null float64
low_rate          1631 non-null float64
dtypes: float64(5), int64(1), object(5)
memory usage: 140.2+ KB


In [ ]:
#gremo od najslabšega načina do najboljšega:

#### Crude looping over DataFrame rows using indices

In [8]:
#1. gremo z zanko čez vse vrstice
# Define a function to manually loop over all rows and return a series of distances
def haversine_looping(df):
    distance_list = []
    for i in range(0, len(df)):
        d = haversine(40.671, -73.985, df.iloc[i]['latitude'], df.iloc[i]['longitude'])
        distance_list.append(d)
        
    return distance_list

In [9]:
%%timeit
# Run the haversine looping function
df['distance'] = haversine_looping(df)

1.02 s ± 6.46 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
#for zanke so najslabša rešitev: 
# for zanka ima zahtevnost O(n^2)
#vgnezdena for zanka ima zahtevnost O(n^3)
#funkcije ki imajo zahtevnost več od kvadratne, običajno niso dobre, so prepočasne

#### Looping with iterrows()

In [11]:
%%timeit
# Haversine applied on rows via iteration
haversine_series = []

for index, row in df.iterrows():
    haversine_series.append(haversine(40.671, -73.985, row['latitude'], row['longitude']))


df['distance'] = haversine_series

342 ms ± 19 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Looping with apply()

In [13]:
%%timeit
# Timing apply on the Haversine function

#apply priredi eno funkcijo vsaki vrstici
#najboljša možnost(najbolj optimizirana),če moramo delati z zankami

df['distance'] = df.apply(lambda row: haversine(40.671, -73.985, row['latitude'], row['longitude']), axis = 1)

151 ms ± 2.85 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
#looping je še vedno slab, če se le da, se mu izognemo

#### Vectorization with Pandas series

In [14]:
%%timeit 
# Vectorized implementation of Haversine applied on Pandas series

#Vektorizacija: več elemntov lahko obdela hkrati
#funkciji podamo kar cel stolpec, numpy potem poskrbi za "loopanje"
df['distance'] = haversine(40.671, -73.985, df['latitude'], df['longitude'])


3.78 ms ± 57.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


####  Vectorization with NumPy arrays

In [15]:
type(df['latitude'].values)

numpy.ndarray

In [16]:
type(df['latitude'])

pandas.core.series.Series

In [17]:
%%timeit
# Vectorized implementation of Haversine applied on NumPy arrays

#v prejšnjem primeru je pretvoril v numpy objekte, sedaj mu pa že podamo numpy, da ne rabi nič pretvarjat
#vse operacije v pythonu se najhitreje izvajajo nad ndarrrayi (nad numpy objekti), hitreje ne gre
df['distance'] = haversine(40.671, -73.985, df['latitude'].values, df['longitude'].values)

491 µs ± 6.28 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
#POVZETEK:
#1. izogni se loopanju
#2. če že moraš loopati, uporabi apply
#3. vektorzacija je hitrejša od loopanja
#4. vektorizacija numpy objektov je najhitrejša

This brings us to a few basic conclusions on optimizing Pandas code:
1. Avoid loops; they’re slow and, in most common use cases, unnecessary.
2. If you must loop, use apply(), not iteration functions.
3. Vectorization is usually better than scalar operations. Most common operations in Pandas can be vectorized.
4. Vector operations on NumPy arrays are more efficient than on native Pandas series.

## I/O Bound Programs

### I/O Bounds

<img src="./images/report_assembly.png">

<img src="./images/report_assembly_bidir.png">

I/O bound tasks are tasks where:
- Our program is reading from an input (like a CSV file).
- Our program is writing to an output (like a text file).
- Our program is waiting for another program to execute something (like a SQL query).
- Our program is waiting for another server to execute something (like an API request).



In [ ]:
#če imaš malo podatkov, se splača vse naenkrat dat v pomnilnik, da samo enkrat beremo iz zunanjega vira

### Profiling an I/O bound task

In [ ]:
#python program se izvaja na enem jedru
#obstajajo knjižnice, s katereimi se python izvaja na več jedrih
#za multiprocessing python ni najboljši


In [18]:
query = '''
SELECT DISTINCT teamID 
FROM Teams 
INNER JOIN TeamsFranchises ON Teams.franchID == TeamsFranchises.franchID 
WHERE TeamsFranchises.active = 'Y';
'''

In [19]:
import cProfile
import sqlite3

conn = sqlite3.connect("data/lahman2015.sqlite")

cur = conn.cursor()
teams = [row[0] for row in cur.execute(query).fetchall()]

In [20]:
print(teams)

['BSN', 'CHN', 'CN2', 'PT1', 'SL4', 'NY1', 'PHI', 'BR3', 'PIT', 'BRO', 'CIN', 'SLN', 'BLA', 'BOS', 'CHA', 'CLE', 'DET', 'MLA', 'PHA', 'WS1', 'SLA', 'NYA', 'ML1', 'BAL', 'KC1', 'LAN', 'SFN', 'LAA', 'MIN', 'WS2', 'HOU', 'NYN', 'CAL', 'ATL', 'OAK', 'KCA', 'SE1', 'MON', 'SDN', 'ML4', 'TEX', 'SEA', 'TOR', 'COL', 'FLO', 'ANA', 'TBA', 'ARI', 'MIL', 'WAS', 'MIA']


In [21]:
import cProfile
import sqlite3

#ekipo po ekipo gremo in appendamo rezultate

query = "SELECT SUM(HR) FROM Batting WHERE teamId=?"
conn = sqlite3.connect("data/lahman2015.sqlite")
cur = conn.cursor()

def calculate_runs(teams):
    home_runs = []
    for team in teams:
        runs = cur.execute(query, [team]).fetchall()
        runs = runs[0][0]
        home_runs.append(runs)
    return home_runs

In [22]:
%%timeit
home_runs = calculate_runs(teams)

102 ms ± 5.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
#profiliramo kodo
profile_string = "home_runs = calculate_runs(teams)"

In [24]:
#cProfile nam pove, kateri del porabi kolko časa
cProfile.run(profile_string)

         157 function calls in 0.119 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.119    0.119 <ipython-input-21-301377ae2693>:10(calculate_runs)
        1    0.000    0.000    0.119    0.119 <string>:1(<module>)
        1    0.000    0.000    0.119    0.119 {built-in method builtins.exec}
       51    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
       51    0.118    0.002    0.118    0.002 {method 'execute' of 'sqlite3.Cursor' objects}
       51    0.000    0.000    0.000    0.000 {method 'fetchall' of 'sqlite3.Cursor' objects}




In [ ]:
#največ časa porabi execute del

### Blocking Tasks

```python
51    0.120    0.002    0.120    0.002 {method 'execute' of 'sqlite3.Cursor' objects}
```

```python
conn = sqlite3.connect(':memory:')
```

In [25]:
import sqlite3

# Create an in memory database.
memory = sqlite3.connect(':memory:')

# Connect to our disk database.
disk = sqlite3.connect('data/lahman2015.sqlite')

# Create a query that will read the contents of the disk database 
# into another database.
dump = ''.join(line for line in disk.iterdump())

# Run the query to copy the database from disk into memory.
memory.executescript(dump)

cur = memory.cursor()

In [26]:
dump[:1000]

'BEGIN TRANSACTION;CREATE TABLE AllstarFull (\nplayerID TEXT,\nyearID INTEGER,\ngameNum INTEGER,\ngameID TEXT,\nteamID TEXT,\nlgID TEXT,\nGP INTEGER,\nstartingPos INTEGER\n);INSERT INTO "AllstarFull" VALUES(\'aaronha01\',1955,0,\'NLS195507120\',\'ML1\',\'NL\',1,NULL);INSERT INTO "AllstarFull" VALUES(\'aaronha01\',1956,0,\'ALS195607100\',\'ML1\',\'NL\',1,NULL);INSERT INTO "AllstarFull" VALUES(\'aaronha01\',1957,0,\'NLS195707090\',\'ML1\',\'NL\',1,9);INSERT INTO "AllstarFull" VALUES(\'aaronha01\',1958,0,\'ALS195807080\',\'ML1\',\'NL\',1,9);INSERT INTO "AllstarFull" VALUES(\'aaronha01\',1959,1,\'NLS195907070\',\'ML1\',\'NL\',1,9);INSERT INTO "AllstarFull" VALUES(\'aaronha01\',1959,2,\'NLS195908030\',\'ML1\',\'NL\',1,9);INSERT INTO "AllstarFull" VALUES(\'aaronha01\',1960,1,\'ALS196007110\',\'ML1\',\'NL\',1,9);INSERT INTO "AllstarFull" VALUES(\'aaronha01\',1960,2,\'ALS196007130\',\'ML1\',\'NL\',1,9);INSERT INTO "AllstarFull" VALUES(\'aaronha01\',1961,1,\'NLS196107110\',\'ML1\',\'NL\',1,NULL

In [27]:
import cProfile
import sqlite3

query = "SELECT SUM(HR) FROM Batting WHERE teamId=?"

def calculate_runs(teams):
    home_runs = []
    for team in teams:
        runs = cur.execute(query, [team]).fetchall()
        runs = runs[0][0]
        home_runs.append(runs)
    return home_runs

In [28]:
profile_string = "home_runs = calculate_runs(teams)"
cProfile.run(profile_string)

         157 function calls in 0.061 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.061    0.061 <ipython-input-27-0ca12a0b9230>:6(calculate_runs)
        1    0.000    0.000    0.061    0.061 <string>:1(<module>)
        1    0.000    0.000    0.061    0.061 {built-in method builtins.exec}
       51    0.000    0.000    0.000    0.000 {method 'append' of 'list' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
       51    0.061    0.001    0.061    0.001 {method 'execute' of 'sqlite3.Cursor' objects}
       51    0.000    0.000    0.000    0.000 {method 'fetchall' of 'sqlite3.Cursor' objects}




### Parallel Execution

<img src="./images/single_threaded.png">

 What if, instead, we could run several queries at once? It might look like this:

<img src="./images/multi_threaded.png">

We can use the Python 3 [threading library](https://docs.python.org/3/library/threading.html) to implement threading in our programs.

In [30]:
def task(team):
    print(team)
    

In [33]:
import threading

thread = threading.Thread(target=task,args=(teams,)) #za teams moramo dat vejico, da ostane tuple!
thread.start()

['BSN', 'CHN', 'CN2', 'PT1', 'SL4', 'NY1', 'PHI', 'BR3', 'PIT', 'BRO', 'CIN', 'SLN', 'BLA', 'BOS', 'CHA', 'CLE', 'DET', 'MLA', 'PHA', 'WS1', 'SLA', 'NYA', 'ML1', 'BAL', 'KC1', 'LAN', 'SFN', 'LAA', 'MIN', 'WS2', 'HOU', 'NYN', 'CAL', 'ATL', 'OAK', 'KCA', 'SE1', 'MON', 'SDN', 'ML4', 'TEX', 'SEA', 'TOR', 'COL', 'FLO', 'ANA', 'TBA', 'ARI', 'MIL', 'WAS', 'MIA']


In [32]:
def task(team):
    print(team)
    
#naredi več threadov, ki ne počakajo vedno, da prejšnji konča
for n, team in enumerate(teams):
    thread = threading.Thread(target=task, args=(team,))
    thread.start()
    #ta print pa čaka, da se prejšnji konča
    print('Started task', n)
    
#ker en print čaka, en pa ne, niammo lepega izpisa

BSN
Started task 0
CHNStarted task 1

CN2
Started task 2
PT1
Started task 3
SL4
Started task 4
NY1Started task 5
PHI
Started task 6
BR3
Started task 7
PIT
Started task 8
BRO
Started task 9
CIN
Started task 10

SLN
Started task 11
BLAStarted task 12

BOS
Started task 13
CHAStarted task 14
CLE
Started task 15

DET
Started task 16
MLAStarted task 17
PHA
Started task 18
WS1
Started task 19

SLA
Started task 20
NYAStarted task 21

ML1
Started task 22
BALStarted task 23

KC1
Started task 24
LANStarted task 25

SFN
Started task 26
LAAStarted task 27

MIN
Started task 28
WS2Started task 29

HOU
Started task 30
NYNStarted task 31

CAL
Started task 32
ATLStarted task 33
OAK
Started task 34

KCA
Started task 35
SE1Started task 36

MON
Started task 37
SDNStarted task 38
ML4
Started task 39

TEX
Started task 40
SEAStarted task 41

TOR
Started task 42
COLStarted task 43

FLO
Started task 44
ANAStarted task 45

TBA
Started task 46
ARIStarted task 47

MIL
Started task 48
WASStarted task 49

MIA
Starte

### Thread Blocking

<img src="./images/three_threads.svg">

In [34]:
import threading
import time

def task(team):
    time.sleep(3)
    print(team)
for n, team in enumerate(teams):
    #naredi novo nit(thread) za team; čaka 3 sekunde, vmes ne rabiš procesorja in zato program nadaljuje (theread se sprosti)
    #pride do spodnjega printa in izpiše
    #nato gre v nov loop, naredi nov thread za novo ekipo, spet spi 3 sekunde, vmes se sprosti in nadaljuje s programom
    # izpiše spodnji print
    #...
    thread = threading.Thread(target=task, args=(team,))
    thread.start()
    
    #ta print ni v threadih, se izpiše takoj
    print('Started task', n)
    
#v pythonu ne moremo hkrati izvajati kode, lahko pa naredimo tako, da nadaljuje takoj, ko se sprosti (neki takega)

Started task 0
Started task 1
Started task 2
Started task 3
Started task 4
Started task 5
Started task 6
Started task 7
Started task 8
Started task 9
Started task 10
Started task 11
Started task 12
Started task 13
Started task 14
Started task 15
Started task 16
Started task 17
Started task 18
Started task 19
Started task 20
Started task 21
Started task 22
Started task 23
Started task 24
Started task 25
Started task 26
Started task 27
Started task 28
Started task 29
Started task 30
Started task 31
Started task 32
Started task 33
Started task 34
Started task 35
Started task 36
Started task 37
Started task 38
Started task 39
Started task 40
Started task 41
Started task 42
Started task 43
Started task 44
Started task 45
Started task 46
Started task 47
Started task 48
Started task 49
Started task 50
CHN
BSN
PT1
CN2
SL4
PHI
NY1
BR3
CIN
BRO
SLN
PIT
BLA
BOS
CHA
DET
WS1
MLA
NYA
LAA
CAL
NYN
LAN
MIN
WS2
ML1
OAK
KCA
MON
SE1
ANA
MIL
SDN
SEA
COL
ML4
BAL
TBA
TOR
TEX
FLO
SFN
SLA
ARI
WAS
HOU
CLE
KC1
AT

### Joining Threads

```python
t1 = threading.Thread(target=task, args=(team,))
t2 = threading.Thread(target=task, args=(team,))
t3 = threading.Thread(target=task, args=(team,))

# Start the first three threads
t1.start()
t2.start()
t3.start()

t1.join() # Wait until t1 finishes.
t2.join() # Wait until t2 finishes.  If it already finished, then keep going.
t3.join() # Wait until t3 finishes.  If it already finished, then keep going.
```

<img src="./images/Screenshot from 2019-06-29 16-40-25.png">

In [35]:
#združujemo threade skupaj
def task(team):
    print(team)

for i in range(11):
    team_names = teams[i*5: (i+1) * 5]
    threads = []
    for team in team_names:
        thread = threading.Thread(target=task, args=(team,))
        thread.start()
        threads.append(thread)
    for thread in threads:
        thread.join()
    print("Finished batch {}".format(i)) 

BSN
CHN
CN2
PT1
SL4
Finished batch 0
NY1
PHI
BR3
PIT
BRO
Finished batch 1
CIN
SLN
BLA
BOS
CHA
Finished batch 2
CLE
DET
MLA
PHA
WS1
Finished batch 3
SLA
NYA
ML1
BALKC1

Finished batch 4
LAN
SFN
LAA
MIN
WS2
Finished batch 5
HOU
NYN
CAL
ATL
OAK
Finished batch 6
KCA
SE1
MON
SDN
ML4
Finished batch 7
TEX
SEA
TOR
COL
FLO
Finished batch 8
ANA
TBA
ARI
MIL
WAS
Finished batch 9
MIA
Finished batch 10


### Locking

It's important to be aware of accessing shared resources when you're working with threads. Some examples of shared resources are:
- The system stdout.
- SQL databases.
- APIs.
- Objects in memory.

```python
naredimo thred in ga zaklenemo:
lock = threading.Lock()

def task(team):
    lock.acquire()
    # This code cannot be executed until a thread acquires the lock.
    print(team)
    lock.release()

t1 = threading.Thread(target=task, args=(team,))
t2 = threading.Thread(target=task, args=(team,))

t1.start()
t2.start()

```

In [36]:
#zaklenemo thread, da ne nadaljuje, dokler nečesa ne konča
import threading
import time
import sys

lock = threading.Lock()

def task(team):
    lock.acquire()
    print(team)
    sys.stdout.flush()
    lock.release()
    
for i in range(11):
    team_names = teams[i*5: (i+1) * 5]
    threads = []
    for team in team_names:
        thread = threading.Thread(target=task, args=(team,))
        thread.start()
        threads.append(thread)
    for thread in threads:
        thread.join()
    print("Finished batch {}".format(i))   

BSN
CHN
CN2
PT1
SL4
Finished batch 0
NY1
PHI
BR3
PIT
BRO
Finished batch 1
CIN
SLN
BOS
CHA
BLA
Finished batch 2
CLE
MLA
PHA
WS1
DET
Finished batch 3
SLA
ML1
NYA
BAL
KC1
Finished batch 4
LAN
LAA
MIN
WS2
SFN
Finished batch 5
HOU
NYN
ATL
CAL
OAK
Finished batch 6
KCA
SE1
MON
SDN
ML4
Finished batch 7
TEX
SEA
TOR
COL
FLO
Finished batch 8
ANA
TBA
ARI
MIL
WAS
Finished batch 9
MIA
Finished batch 10


### Thread Safety

In general, these operations are not thread safe:
- Modifying data in memory.
- Writing to a file.
- Adding data to a database.
- Modifying data via API.


In [37]:
import cProfile
import sqlite3
import threading
import sys

In [38]:
query = "SELECT DISTINCT teamID from Teams inner join TeamsFranchises on Teams.franchID == TeamsFranchises.franchID where TeamsFranchises.active = 'Y';"

In [39]:
conn = sqlite3.connect("data/lahman2015.sqlite", check_same_thread=False)

In [40]:
cur = conn.cursor()
teams = [row[0] for row in cur.execute(query).fetchall()]

query = "SELECT SUM(HR) FROM Batting WHERE teamId=?"
lock = threading.Lock()

In [41]:
def calculate_runs(team):
    cur = conn.cursor()
    runs = cur.execute(query, [team]).fetchall()
    runs = runs[0][0]
    lock.acquire()
    print(team, ':', runs)
    sys.stdout.flush()
    lock.release()
    return runs


threads = []

for team in teams:
    thread = threading.Thread(target=calculate_runs, args=(team,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

BSN : 3424
CN2 : 267
SL4 : 305
CHN : 13530
PT1 : 54
NY1 : 5777
PHI : 12503
BRO : 4336
BR3 : 143
PIT : 10878
CIN : 12383
BLA : 57
CHA : 10792
SLN : 11157
BOS : 12883
PHA : 3502
SLA : 3014
WS1 : 2786
MLA : 26
DET : 13160
CLE : 12333
NYA : 15218
ML1 : 2230
KC1 : 1480
LAN : 7601
SFN : 8348
NYN : 6817
MIN : 7393
OAK : 7438
BAL : 9592
LAA : 2276
WS2 : 1387
SDN : 5648
ATL : 7535
HOU : 6536
KCA : 5613
SE1 : 125
MON : 4381
CAL : 3912
TEX : 7055
SEA : 5976
ML4 : 3664
TOR : 6415
COL : 4120
FLO : 2816
ANA : 1324
TBA : 2823
ARI : 2987
MIL : 3160
WAS : 2002
MIA : 474


## Optimizing Python Code with pandas

### Basic Looping

### Select columns and rows efficiently


In [42]:
data = pd.read_csv('data/school.csv')
data.head(3)

,School ID,School Name,Building Code,Street Address,City,State,Zip Code
0,02M260,Clinton School Writers and Artists,M933,425 West 33rd Street,Manhattan,NY,10001
1,06M211,Inwood Early College for Health and Informatio...,M052,650 Academy Street,Manhattan,NY,10002
2,01M539,"New Explorations into Science, Technology and ...",M022,111 Columbia Street,Manhattan,NY,10002


In [43]:
data['City'].value_counts().head(10)

Brooklyn            121
Bronx               118
Manhattan           106
Jamaica              13
Long Island City     12
Staten Island        10
Flushing              8
Astoria               6
Elmhurst              5
Rockaway Park         4
Name: City, dtype: int64

In [44]:
# save the top cities in a list
top_cities = ['Brooklyn','Bronx','Manhattan','Jamaica','Long Island City']

In [48]:
%%timeit

#napiši kodo, ki bo vse 'City', ki niso v top_cities preimenovala v Other
#način 1:
data['City'] = data['City'].where(data['City'].isin(top_cities), 'Other')



1.25 ms ± 121 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [50]:
%%timeit
#način 2:
data.loc[data['City'].isin(top_cities) == False, 'City'] = 'Other'

4 ms ± 145 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [51]:
data.City.value_counts()

Brooklyn            121
Bronx               118
Manhattan           106
Other                65
Jamaica              13
Long Island City     12
Name: City, dtype: int64

In [52]:
data = pd.read_csv('data/school.csv')

In [54]:
%%timeit

#slaba praksa
data['City'][(data['City'].isin(top_cities) == False)] = 'Other'

/home/analitika2/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


361 ms ± 11.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [55]:
data.City.value_counts()

Brooklyn            121
Bronx               118
Manhattan           106
Other                65
Jamaica              13
Long Island City     12
Name: City, dtype: int64

### Uporaba biult-in funkciji

In [ ]:
#najbolj se splača uporabljati built-in funkcije, ker so najbolj optimizirane

### Joining on indexes is faster than joining on columns

Construct some sample data:

In [56]:
n = 100000

i1 = np.arange(n)
np.random.shuffle(i1)
df1 = pd.DataFrame({'i': i1,
                    'j': np.random.randint(1,1000,n),
                    'k': np.random.randint(1,1000,n)})

i2 = np.arange(n)
np.random.shuffle(i1)
df2 = pd.DataFrame({'i': i2,
                    'm': np.random.randint(1,1000,n),
                    'n': np.random.randint(1,1000,n)})

In [57]:
df1.head()

,i,j,k
0,97382,161,699
1,8316,310,482
2,28153,841,703
3,4762,616,250
4,99615,754,125


In [58]:
df2.head()

,i,m,n
0,0,100,814
1,1,485,621
2,2,579,253
3,3,758,360
4,4,680,807


In [59]:
%%timeit
df1.merge(df2, on = 'i')

18.9 ms ± 2.71 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [60]:
#merganje po indeksu je precej hitreje kot merganje po stolpcu

#določimo indekse
df1 = df1.set_index('i')
df2 = df2.set_index('i')

In [61]:
%%timeit

#merganje po indeksu je precej hitreje kot merganje po stolpcu
df1.merge(df2, left_index = True, right_index= True)

9.41 ms ± 491 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## PRIMER: Pohitritev pandas kode

Vir: 
- [Fast, Flexible, Easy and Intuitive: How to Speed Up Your Pandas Projects](https://realpython.com/fast-flexible-pandas/)

### Naloga

Računali bomo ceno elektrike:
podatke imamo za celo leto.
Imamo 3 tarife: vsako uro moramo pomnožit s ceno tarife, kamor spada ura

### Priprava podatkov

In [62]:
import pandas as pd

In [63]:
df = pd.read_csv('data/demand_profile.csv')

In [64]:
df.head()

,date_time,energy_kwh
0,1/1/13 0:00,0.586
1,1/1/13 1:00,0.580
2,1/1/13 2:00,0.572
3,1/1/13 3:00,0.596
4,1/1/13 4:00,0.592


In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 2 columns):
date_time     8760 non-null object
energy_kwh    8760 non-null float64
dtypes: float64(1), object(1)
memory usage: 137.0+ KB


In [66]:
df.dtypes

date_time      object
energy_kwh    float64
dtype: object

In [67]:
type(df.iloc[0, 0])

str

In [68]:
#date_time je tipa object, pretvorimo ga v datetime

df['date_time'] = pd.to_datetime(df['date_time'])
df['date_time'].dtype

dtype('<M8[ns]')

In [69]:
df.head()

,date_time,energy_kwh
0,2013-01-01 00:00:00,0.586
1,2013-01-01 01:00:00,0.580
2,2013-01-01 02:00:00,0.572
3,2013-01-01 03:00:00,0.596
4,2013-01-01 04:00:00,0.592


In [71]:
def convert(df, column_name):
    return pd.to_datetime(df[column_name])

df = pd.read_csv('data/demand_profile.csv')
df_coverted = df.copy()

In [72]:
%%timeit -r 3 -n 10
df_coverted['date_time'] = convert(df, 'date_time')

1.51 s ± 30.5 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [74]:
#uporabimo convert, ki že ima določen format - precej hitreje je!
def convert_with_format(df, column_name):
    return pd.to_datetime(df[column_name], format='%d/%m/%y %H:%M')

In [75]:
%%timeit -r 3 -n 10
df_coverted['date_time'] = convert_with_format(df, 'date_time')

61.2 ms ± 365 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


In [76]:
859/18

47.72222222222222

In [77]:
df_coverted.head()

,date_time,energy_kwh
0,2013-01-01 00:00:00,0.586
1,2013-01-01 01:00:00,0.580
2,2013-01-01 02:00:00,0.572
3,2013-01-01 03:00:00,0.596
4,2013-01-01 04:00:00,0.592


In [78]:
df_coverted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 2 columns):
date_time     8760 non-null datetime64[ns]
energy_kwh    8760 non-null float64
dtypes: datetime64[ns](1), float64(1)
memory usage: 137.0 KB


In [ ]:
#gremo na več načinov: od najslabšega do najboljšega

### 1) Simple Looping Over Pandas Data

<table class="table table-hover">
<thead>
<tr>
<th>Tariff Type</th>
<th>Cents per kWh</th>
<th>Time Range</th>
</tr>
</thead>
<tbody>
<tr>
<td>Peak</td>
<td>28</td>
<td>17:00 to 24:00</td>
</tr>
<tr>
<td>Shoulder</td>
<td>20</td>
<td>7:00 to 17:00</td>
</tr>
<tr>
<td>Off-Peak</td>
<td>12</td>
<td>0:00 to 7:00</td>
</tr>
</tbody>
</table>

In [79]:
#če bi bila cena v vsaki uri 28
df_test = df_coverted.copy()
df_test['cost_cents'] = df['energy_kwh'] * 28

In [82]:
df_test['cost_cents'].sum()/1000

160.31509199999996

In [80]:
df_test.head()

,date_time,energy_kwh,cost_cents
0,2013-01-01 00:00:00,0.586,16.408
1,2013-01-01 01:00:00,0.580,16.240
2,2013-01-01 02:00:00,0.572,16.016
3,2013-01-01 03:00:00,0.596,16.688
4,2013-01-01 04:00:00,0.592,16.576


In [83]:
def apply_tariff(kwh, hour):
    """Calculates cost of electricity for given hour."""    
    if 0 <= hour < 7:
        rate = 12
    elif 7 <= hour < 17:
        rate = 20
    elif 17 <= hour < 24:
        rate = 28
    else:
        raise ValueError(f'Invalid hour: {hour}')
    return rate * kwh

In [92]:
# NOTE: Don't do this!
def apply_tariff_loop(df):
    """Calculate costs in loop.  Modifies `df` inplace."""
    energy_cost_list = []
    for i in range(len(df)):
        energy_cost_list.append(apply_tariff(df.iloc[i]['energy_kwh'], df.iloc[i]['date_time'].hour))
        
    df['cost_cents'] = energy_cost_list

In [93]:
%%timeit -r 3 -n 10

#zeloo počasi, slaba rešitev!!
apply_tariff_loop(df_coverted)

5.08 s ± 180 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


### 2) Looping with .itertuples() and .iterrows()

In [94]:
for index, row in df[:5].iterrows():
    print(index)
    print(row)
    print('energy_kwh' ,row['energy_kwh'])
    print('-------')

0
date_time     1/1/13 0:00
energy_kwh          0.586
Name: 0, dtype: object
energy_kwh 0.586
-------
1
date_time     1/1/13 1:00
energy_kwh           0.58
Name: 1, dtype: object
energy_kwh 0.58
-------
2
date_time     1/1/13 2:00
energy_kwh          0.572
Name: 2, dtype: object
energy_kwh 0.5720000000000001
-------
3
date_time     1/1/13 3:00
energy_kwh          0.596
Name: 3, dtype: object
energy_kwh 0.596
-------
4
date_time     1/1/13 4:00
energy_kwh          0.592
Name: 4, dtype: object
energy_kwh 0.5920000000000001
-------


In [96]:
def apply_tariff_iterrows(df):
    
    energy_cost_list = []
    for index, row in df.iterrows():
        energy_cost_list.append(apply_tariff(row['energy_kwh'], row['date_time'].hour))
        
    df['cost_cents'] = energy_cost_list

In [97]:
%%timeit 

#precej hitreje kot prva rešitev
apply_tariff_iterrows(df_coverted)

1.52 s ± 173 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 3) Pandas’ .apply()

In [98]:
def apply_tariff_withapply(df):
    df['cost_cents'] = df.apply(lambda row: apply_tariff(row['energy_kwh'], row['date_time'].hour), axis = 1)


In [99]:
%%timeit 
apply_tariff_withapply(df_coverted)

393 ms ± 4.52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 4) Selecting Data With .isin()

In [101]:

#nastavimo date_time za index
df_coverted = df.copy()
df_coverted['date_time'] = convert_with_format(df, 'date_time')
df_coverted.set_index('date_time', inplace=True)

In [102]:
df_coverted.head()

,energy_kwh
date_time,
2013-01-01 00:00:00,0.586
2013-01-01 01:00:00,0.580
2013-01-01 02:00:00,0.572
2013-01-01 03:00:00,0.596
2013-01-01 04:00:00,0.592


In [107]:
#Izbermeo ven intervale in vsak interval pomnožimo z ustrezno ceno
def apply_tariff_isin(df):
    peak_hours = df.index.hour.isin(range(17,24))
    shoulder_hours = df.index.hour.isin(range(7, 17))
    offpeak_hours = df.index.hour.isin(range(0,7))

    df.loc[peak_hours, 'cost_cents'] = df.loc[peak_hours, 'energy_kwh'] * 28
    df.loc[shoulder_hours, 'cost_cents'] = df.loc[shoulder_hours, 'energy_kwh'] * 20
    df.loc[offpeak_hours, 'cost_cents'] = df.loc[offpeak_hours, 'energy_kwh'] * 12
   

In [108]:
%%timeit 
apply_tariff_isin(df_coverted)

8.37 ms ± 556 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### 5) Pandas’ pd.cut() function

In [109]:
#pd.cut()

df_coverted = df.copy()
df_coverted['date_time'] = convert_with_format(df, 'date_time')
df_coverted.set_index('date_time', inplace=True)

> **[pandas.cut](https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.cut.html)**
- `pandas.cut(x, bins, right=True, labels=None, retbins=False, precision=3, include_lowest=False, duplicates='raise')`
- Bin values into discrete intervals.
- Use cut when you need to segment and sort data values into bins. This function is also useful for going from a continuous variable to a categorical variable. For example, cut could convert ages to groups of age ranges. Supports binning into an equal number of bins, or a pre-specified array of bins.

In [112]:
df_coverted.index.hour


Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9,
            ...
            14, 15, 16, 17, 18, 19, 20, 21, 22, 23],
           dtype='int64', name='date_time', length=8760)

In [113]:
#naredimo časovno serijo cen, ki jo bomo nato pomnožili s porabo
pd.cut(x = df_coverted.index.hour,
      bins = [0,7,17,24],
      include_lowest=True,
      labels = [12,20,28])

[12, 12, 12, 12, 12, ..., 28, 28, 28, 28, 28]
Length: 8760
Categories (3, int64): [12 < 20 < 28]

In [115]:
def apply_tariff_cut(df):
    cents_per_kwh = pd.cut(x = df_coverted.index.hour,
          bins = [0,7,17,24],
          include_lowest=True,
          labels = [12,20,28])
    df['cost_cents'] = cents_per_kwh * df['energy_kwh']


In [116]:
%%timeit -r 3 -n 10
apply_tariff_cut(df_coverted)

3.46 ms ± 152 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


### 6) Using NumPy

In [117]:
import numpy as np

In [118]:
df_coverted = df.copy()
df_coverted['date_time'] = convert_with_format(df, 'date_time')
df_coverted.set_index('date_time', inplace=True)

> **[numpy.digitize](https://docs.scipy.org/doc/numpy/reference/generated/numpy.digitize.html)**
- `numpy.digitize(x, bins, right=False)`
- Return the indices of the bins to which each value in input array belongs.

In [119]:
#numpy digitize: računa po intervalih (podobno kot v primeru 5)
np.digitize(df_coverted.index.hour.values, 
           bins = [7,17,24])

#dobimo časovno vrsto 0/1/2, zamenjati moramo s cenami

array([0, 0, 0, ..., 2, 2, 2])

In [120]:
def apply_tariff_digitize(df):
    prices = np.array([12,20,28])
    bins = np.digitize(df_coverted.index.hour.values, bins = [7,17,24])
    df['cost_cents'] = prices[bins] * df['energy_kwh'].values


In [121]:
%%timeit -r 3 -n 10
apply_tariff_digitize(df_coverted)

1.22 ms ± 171 µs per loop (mean ± std. dev. of 3 runs, 10 loops each)


### Prevent Reprocessing with HDFStore

[What is HDF5](https://portal.hdfgroup.org/display/knowledge/What+is+HDF5)

In [ ]:
#omogoča, da v csv shranimo dataframe z ohranjenimi podatkovnimi tipi (takimi, kot smo jih določili
#data frame-u) - shrani se točno tak df, kot je, ne izgubijo se podatkovni tipi

In [122]:
df_coverted.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8760 entries, 2013-01-01 00:00:00 to 2013-12-31 23:00:00
Data columns (total 2 columns):
energy_kwh    8760 non-null float64
cost_cents    8760 non-null float64
dtypes: float64(2)
memory usage: 205.3 KB


In [123]:
# Create storage object with filename `processed_data`
data_store = pd.HDFStore('data/OUT_processed_data.h5')

# Put DataFrame into the object setting the key as 'preprocessed_df'
data_store['preprocessed_df'] = df_coverted
data_store.close()

In [124]:
data_store = pd.HDFStore('data/OUT_processed_data.h5')

# Retrieve data using key
preprocessed_df = data_store['preprocessed_df']
data_store.close()

In [125]:
preprocessed_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8760 entries, 2013-01-01 00:00:00 to 2013-12-31 23:00:00
Data columns (total 2 columns):
energy_kwh    8760 non-null float64
cost_cents    8760 non-null float64
dtypes: float64(2)
memory usage: 205.3 KB


### Povzetek

<ul>
<li>
<p>Try to use <a href="https://realpython.com/numpy-array-programming/#what-is-vectorization">vectorized operations</a> where possible rather than approaching problems with the <code>for x in df...</code> mentality. If your code is home to a lot of for-loops, it might be better suited to working with native Python data structures, because Pandas otherwise comes with a lot of overhead.</p>
</li>
<li>
<p>If you have more complex operations where vectorization is simply impossible or too difficult to work out efficiently, use the <code>.apply()</code> method.</p>
</li>
<li>
<p>If you do have to loop over your array (which does happen), use <code>.iterrows()</code> or <code>.itertuples()</code> to improve speed and syntax.</p>
</li>
<li>
<p>Pandas has a lot of optionality, and there are almost always several ways to get from A to B. Be mindful of this, compare how different routes perform, and choose the one that works best in the context of your project.</p>
</li>
<li>
<p>Once you’ve got a data cleaning script built, avoid reprocessing by storing your intermediate results with HDFStore.</p>
</li>
<li>
<p>Integrating NumPy into Pandas operations can often improve speed and simplify syntax.</p>
</li>
</ul>

## Drugi nasveti

###  [Numba](https://numba.pydata.org/)

Numba translates Python functions to optimized machine code at runtime using the industry-standard LLVM compiler library. Numba-compiled numerical algorithms in Python can approach the speeds of C or FORTRAN.

### pandas.eval() for Efficient Operations

[Dokumentacija](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.eval.html)

[High-Performance Pandas: eval() and query()](https://jakevdp.github.io/PythonDataScienceHandbook/03.12-performance-eval-and-query.html#pandas.eval()-for-Efficient-Operations)

As of version 0.13 (released January 2014), Pandas includes some experimental tools that allow you to directly access C-speed operations without costly allocation of intermediate arrays. These are the eval() and query() functions, which rely on the Numexpr package.